### Table of Contents

1. **Libraries and Functions**

2. **Regulatory Parameters**
   - Regulatory Constants
   - Supervisory Risk Weights
   - SNH Correlations

3. **Data**
   - Portfolio Data
   - Single Name Hedges
   - Index Hedges

4. **Standalone SVA (SCVA)**
   - Alpha
   - Counterparty Risk Weights
   - Maturity
   - Discount Factor
   - EAD
   - SCVA

5. **K Reduced**

6. **K Hedged**
   - Single Name Hedges (SNH)
     - Correlation Factor
     - Hedge Risk Weights
     - Maturity
     - Discount Factor
     - Notional
     - SNH
   - Hedge Mismatch Adjustment (HMA)
   - Index Hedges (IH)
     - Index Risk Weights
     - Scalar
     - Maturity
     - Discount Factor
     - Notional
     - IH
   - K Hedged
     - Systematic Component
     - Idiosyncratic Component
     - Hedge Mismatch Total
     - K Hedged Calculation

7. **K Full**

8. **K Final**

### Libraries and Functions

In [3]:
# import necessary libraries
import numpy as np
import pandas as pd
from tabulate import tabulate

In [4]:
# ============================================================
# FORMAT TABLE FUNCTION
# ============================================================

def format_table(df, columns, col_formats=None, alignments=None):
    """
    Format a DataFrame for display using tabulate.

    Parameters:
        df: pandas DataFrame
        columns: list of column names to display
        col_formats: dict mapping column names to format strings (e.g., ',.0f', ',.2f', '.4f')
        alignments: tuple of alignment strings ('left', 'right', 'center')

    Returns:
        Formatted table string
    """
    df_copy = df[columns].copy()

    if col_formats:
        for col, fmt in col_formats.items():
            if col in df_copy.columns:
                df_copy[col] = df_copy[col].apply(
                    lambda x: f"{x:{fmt}}" if isinstance(x, (int, float, np.integer, np.floating)) else x
                )

    return tabulate(
        df_copy,
        headers='keys',
        tablefmt='simple_outline',
        showindex=False,
        colalign=alignments
    )

### Regulatory Parameters

#### Regulatory Constants

In [5]:
# ============================================================
# REGULATORY CONSTANTS
# ============================================================

# Supervisory scalars from PRA PS9/24
DS_BA_CVA = 0.65          # Discount scalar applied to final capital
RHO = 0.50                # Correlation for systematic/idiosyncratic split
BETA = 0.25               # Hedge recognition floor (min 25% of K_reduced retained)
ALPHA_STANDARD = 1.4      # Alpha for most counterparties
ALPHA_PENSION = 1.0       # Alpha for pension funds
SUPERVISORY_RATE = 0.05   # Discount rate in DF formula
INDEX_SCALAR = 0.70       # Multiplier for index hedge risk weights

# Create DataFrame for display
df_reg_constants = pd.DataFrame({
    "Item": [
        "DS_BA_CVA",
        "RHO (ρ)",
        "BETA (β)",
        "ALPHA_STANDARD (α)",
        "ALPHA_PENSION (α)",
        "SUPERVISORY_RATE",
        "INDEX_SCALAR"
    ],
    "Value": [
        DS_BA_CVA,
        RHO,
        BETA,
        ALPHA_STANDARD,
        ALPHA_PENSION,
        SUPERVISORY_RATE,
        INDEX_SCALAR
    ],
    "Description": [
        "Discount scalar applied to final capital",
        "Correlation for systematic/idiosyncratic split",
        "Hedge recognition floor (min 25% of K_reduced retained)",
        "Alpha multiplier for standard counterparties",
        "Alpha multiplier for pension fund counterparties",
        "Discount rate used in DF formula (5%)",
        "Multiplier for index hedge risk weights"
    ]
})

# Display table
print(format_table(
    df_reg_constants,
    columns=["Item", "Value", "Description"],
    col_formats={"Value": ".2f"},
    alignments=("left", "right", "left")
))

┌────────────────────┬─────────┬─────────────────────────────────────────────────────────┐
│ Item               │   Value │ Description                                             │
├────────────────────┼─────────┼─────────────────────────────────────────────────────────┤
│ DS_BA_CVA          │    0.65 │ Discount scalar applied to final capital                │
│ RHO (ρ)            │     0.5 │ Correlation for systematic/idiosyncratic split          │
│ BETA (β)           │    0.25 │ Hedge recognition floor (min 25% of K_reduced retained) │
│ ALPHA_STANDARD (α) │     1.4 │ Alpha multiplier for standard counterparties            │
│ ALPHA_PENSION (α)  │       1 │ Alpha multiplier for pension fund counterparties        │
│ SUPERVISORY_RATE   │    0.05 │ Discount rate used in DF formula (5%)                   │
│ INDEX_SCALAR       │     0.7 │ Multiplier for index hedge risk weights                 │
└────────────────────┴─────────┴─────────────────────────────────────────────────────────┘

#### Supervisory Risk Weights

In [6]:
# ============================================================
# RISK WEIGHTS BY SECTOR AND CREDIT QUALITY
# ============================================================

# Risk weights by sector and credit quality (Table 1)
RISK_WEIGHTS = {
    "SOV": {"IG": 0.005, "HY": 0.020},
    "GOV": {"IG": 0.010, "HY": 0.040},
    "FIN": {"IG": 0.050, "HY": 0.120},
    "PEN": {"IG": 0.035, "HY": 0.085},
    "IND": {"IG": 0.030, "HY": 0.070},
    "CON": {"IG": 0.030, "HY": 0.085},
    "TEC": {"IG": 0.020, "HY": 0.055},
    "HLT": {"IG": 0.015, "HY": 0.050},
    "OTH": {"IG": 0.050, "HY": 0.120},
}

# Create DataFrame for display
df_risk_weights = pd.DataFrame({
    "Industry": [
        "SOV - Sovereigns",
        "GOV - Government Entities",
        "FIN - Financials",
        "PEN - Pension Funds",
        "IND - Industrials",
        "CON - Consumer",
        "TEC - Technology",
        "HLT - Healthcare",
        "OTH - Other"
    ],
    "IG": [
        RISK_WEIGHTS["SOV"]["IG"],
        RISK_WEIGHTS["GOV"]["IG"],
        RISK_WEIGHTS["FIN"]["IG"],
        RISK_WEIGHTS["PEN"]["IG"],
        RISK_WEIGHTS["IND"]["IG"],
        RISK_WEIGHTS["CON"]["IG"],
        RISK_WEIGHTS["TEC"]["IG"],
        RISK_WEIGHTS["HLT"]["IG"],
        RISK_WEIGHTS["OTH"]["IG"]
    ],
    "HY": [
        RISK_WEIGHTS["SOV"]["HY"],
        RISK_WEIGHTS["GOV"]["HY"],
        RISK_WEIGHTS["FIN"]["HY"],
        RISK_WEIGHTS["PEN"]["HY"],
        RISK_WEIGHTS["IND"]["HY"],
        RISK_WEIGHTS["CON"]["HY"],
        RISK_WEIGHTS["TEC"]["HY"],
        RISK_WEIGHTS["HLT"]["HY"],
        RISK_WEIGHTS["OTH"]["HY"]
    ]
})

# Display table
print(format_table(
    df_risk_weights,
    columns=["Industry", "IG", "HY"],
    col_formats={"IG": ".3f", "HY": ".3f"},
    alignments=("left", "right", "right")
))

┌───────────────────────────┬───────┬───────┐
│ Industry                  │    IG │    HY │
├───────────────────────────┼───────┼───────┤
│ SOV - Sovereigns          │ 0.005 │  0.02 │
│ GOV - Government Entities │  0.01 │  0.04 │
│ FIN - Financials          │  0.05 │  0.12 │
│ PEN - Pension Funds       │ 0.035 │ 0.085 │
│ IND - Industrials         │  0.03 │  0.07 │
│ CON - Consumer            │  0.03 │ 0.085 │
│ TEC - Technology          │  0.02 │ 0.055 │
│ HLT - Healthcare          │ 0.015 │  0.05 │
│ OTH - Other               │  0.05 │  0.12 │
└───────────────────────────┴───────┴───────┘


#### SNH Correlations

In [7]:
# ============================================================
# HEDGE CORRELATIONS (r_hc)
# ============================================================

# Hedge correlations (Table 2)
HEDGE_CORRELATIONS = {
    "direct": 1.00,
    "legally_related": 0.80,
    "sector_region": 0.50,
}

# Create DataFrame for display
df_hedge_correlations = pd.DataFrame({
    "Hedge Type": [
        "Direct",
        "Legally Related",
        "Sector/Region"
    ],
    "Correlation (r_hc)": [
        HEDGE_CORRELATIONS["direct"],
        HEDGE_CORRELATIONS["legally_related"],
        HEDGE_CORRELATIONS["sector_region"]
    ],
    "Description": [
        "Direct reference to counterparty",
        "Legally related entity (parent/subsidiary)",
        "Same sector and region (proxy hedge)"
    ]
})

# Display table
print(format_table(
    df_hedge_correlations,
    columns=["Hedge Type", "Correlation (r_hc)", "Description"],
    col_formats={"Correlation (r_hc)": ".2f"},
    alignments=("left", "right", "left")
))

┌─────────────────┬──────────────────────┬────────────────────────────────────────────┐
│ Hedge Type      │   Correlation (r_hc) │ Description                                │
├─────────────────┼──────────────────────┼────────────────────────────────────────────┤
│ Direct          │                    1 │ Direct reference to counterparty           │
│ Legally Related │                  0.8 │ Legally related entity (parent/subsidiary) │
│ Sector/Region   │                  0.5 │ Same sector and region (proxy hedge)       │
└─────────────────┴──────────────────────┴────────────────────────────────────────────┘


### Data

#### Portfolio Data

In [12]:
# ============================================================
# PORTFOLIO DATA (COUNTERPARTIES & NETTING SETS)
# ============================================================

# Step 1: Create counterparty master data
df_counterparties = pd.DataFrame({
    "counterparty_id": ["CP001", "CP002", "CP003", "CP004", "CP005"],
    "counterparty_name": [
        "Alpha Bank Ltd",
        "Northern Energy Corp",
        "Maple Pension Trust",
        "GlobalTech Solutions",
        "Riverside Finance Ltd"
    ],
    "sector_code": ["FIN", "IND", "PEN", "TEC", "FIN"],
    "credit_quality": ["IG", "IG", "IG", "HY", "IG"],
})

# Step 2: Look up risk weight from RISK_WEIGHTS table
df_counterparties["risk_weight"] = df_counterparties.apply(
    lambda r: RISK_WEIGHTS[r["sector_code"]][r["credit_quality"]], axis=1
)

# Step 3: Assign alpha (1.0 for pension funds, 1.4 for others)
df_counterparties["alpha"] = np.where(
    df_counterparties["sector_code"] == "PEN", 
    ALPHA_PENSION, 
    ALPHA_STANDARD
)

# Step 4: Create netting set data
df_netting_sets = pd.DataFrame({
    "netting_set_id": ["NS001", "NS002", "NS003", "NS004", "NS005"],
    "counterparty_id": ["CP001", "CP002", "CP003", "CP004", "CP005"],
    "netting_set_name": [
        "Alpha Bank - IRS Portfolio",
        "Northern Energy - Commodity Hedges",
        "Maple Pension - LDI Swaps",
        "GlobalTech - FX Forwards",
        "Riverside - Repo Book"
    ],
    "exposure_method": ["IMM", "SA-CCR", "IMM", "SA-CCR", "SFT"],
    "ead_ns": [1500, 450, 800, 250, 2000],  # £000s
    "m_ns": [4.85, 1.61, 18.50, 1.00, 0.10]  # Years
})

# Step 5: Combine into single portfolio DataFrame
df_portfolio = df_netting_sets.merge(
    df_counterparties[["counterparty_id", "counterparty_name", "sector_code", 
                       "credit_quality", "risk_weight", "alpha"]],
    on="counterparty_id"
)

# Display combined portfolio table (without alpha - shown later)
print(format_table(
    df_portfolio,
    columns=["netting_set_id", "counterparty_name", "sector_code", "credit_quality",
             "risk_weight", "exposure_method", "ead_ns", "m_ns"],
    col_formats={"risk_weight": ".4f", "ead_ns": ",.0f", "m_ns": ".2f"},
    alignments=("left", "left", "center", "center", "right", "center", "right", "right")
))

┌──────────────────┬───────────────────────┬───────────────┬──────────────────┬───────────────┬───────────────────┬──────────┬────────┐
│ netting_set_id   │ counterparty_name     │  sector_code  │  credit_quality  │   risk_weight │  exposure_method  │   ead_ns │   m_ns │
├──────────────────┼───────────────────────┼───────────────┼──────────────────┼───────────────┼───────────────────┼──────────┼────────┤
│ NS001            │ Alpha Bank Ltd        │      FIN      │        IG        │          0.05 │        IMM        │    1,500 │   4.85 │
│ NS002            │ Northern Energy Corp  │      IND      │        IG        │          0.03 │      SA-CCR       │      450 │   1.61 │
│ NS003            │ Maple Pension Trust   │      PEN      │        IG        │         0.035 │        IMM        │      800 │   18.5 │
│ NS004            │ GlobalTech Solutions  │      TEC      │        HY        │         0.055 │      SA-CCR       │      250 │      1 │
│ NS005            │ Riverside Finance Ltd │    

#### Single Name Hedges

In [10]:
# ============================================================
# SINGLE NAME HEDGE DATA
# ============================================================

# Create single-name hedge data
df_sn_hedges = pd.DataFrame({
    "hedge_id": ["SN001", "SN002", "SN003", "SN004", "SN005"],
    "hedged_counterparty_id": ["CP001", "CP001", "CP002", "CP004", "CP005"],
    "hedge_reference_entity": [
        "Alpha Bank Ltd",              # Direct reference
        "Alpha Asset Management",      # Subsidiary (legally related)
        "Atlantic Oil Co",             # Same sector/region proxy
        "GlobalTech Holdings Inc",     # Parent (legally related)
        "Riverside Finance Ltd"        # Direct reference
    ],
    "hedge_type": ["direct", "legally_related", "sector_region", "legally_related", "direct"],
    "reference_sector": ["FIN", "FIN", "IND", "TEC", "FIN"],
    "reference_cq": ["IG", "IG", "IG", "HY", "IG"],
    "notional_b_sn": [500, 300, 200, 150, 1000],  # B_h^SN in £000s
    "maturity_m_sn": [3.0, 2.5, 2.0, 1.5, 0.5]    # M_h^SN in years
})

# Display single name hedge table
print(format_table(
    df_sn_hedges,
    columns=["hedge_id", "hedged_counterparty_id", "hedge_reference_entity", 
             "hedge_type", "reference_sector", "reference_cq", 
             "notional_b_sn", "maturity_m_sn"],
    col_formats={"notional_b_sn": ",.0f", "maturity_m_sn": ".2f"},
    alignments=("left", "center", "left", "left", "center", "center", "right", "right")
))

┌────────────┬──────────────────────────┬──────────────────────────┬─────────────────┬────────────────────┬────────────────┬─────────────────┬─────────────────┐
│ hedge_id   │  hedged_counterparty_id  │ hedge_reference_entity   │ hedge_type      │  reference_sector  │  reference_cq  │   notional_b_sn │   maturity_m_sn │
├────────────┼──────────────────────────┼──────────────────────────┼─────────────────┼────────────────────┼────────────────┼─────────────────┼─────────────────┤
│ SN001      │          CP001           │ Alpha Bank Ltd           │ direct          │        FIN         │       IG       │             500 │               3 │
│ SN002      │          CP001           │ Alpha Asset Management   │ legally_related │        FIN         │       IG       │             300 │             2.5 │
│ SN003      │          CP002           │ Atlantic Oil Co          │ sector_region   │        IND         │       IG       │             200 │               2 │
│ SN004      │          CP004     

#### Index Hedges

In [11]:
# ============================================================
# INDEX HEDGE DATA
# ============================================================

# Create index hedge data
df_index_hedges = pd.DataFrame({
    "index_id": ["IH001", "IH002"],
    "index_name": ["EuroCredit IG Index", "NorthAm IG Index"],
    "notional_b_ind": [750, 500],        # B_ind in £000s
    "maturity_m_ind": [5.0, 3.0]         # M_ind in years
})

# Display index hedge table
print(format_table(
    df_index_hedges,
    columns=["index_id", "index_name", "notional_b_ind", "maturity_m_ind"],
    col_formats={"notional_b_ind": ",.0f", "maturity_m_ind": ".2f"},
    alignments=("left", "left", "right", "right")
))

┌────────────┬─────────────────────┬──────────────────┬──────────────────┐
│ index_id   │ index_name          │   notional_b_ind │   maturity_m_ind │
├────────────┼─────────────────────┼──────────────────┼──────────────────┤
│ IH001      │ EuroCredit IG Index │              750 │                5 │
│ IH002      │ NorthAm IG Index    │              500 │                3 │
└────────────┴─────────────────────┴──────────────────┴──────────────────┘


### Standalone SVA (SCVA)

#### Alpha

In [13]:
# ============================================================
# ALPHA CALCULATION
# ============================================================

# Create alpha table showing counterparty, sector, and alpha
df_alpha = df_counterparties[["counterparty_name", "sector_code", "alpha"]].copy()

# Display alpha table
print(format_table(
    df_alpha,
    columns=["counterparty_name", "sector_code", "alpha"],
    col_formats={"alpha": ".2f"},
    alignments=("left", "center", "right")
))

┌───────────────────────┬───────────────┬─────────┐
│ counterparty_name     │  sector_code  │   alpha │
├───────────────────────┼───────────────┼─────────┤
│ Alpha Bank Ltd        │      FIN      │     1.4 │
│ Northern Energy Corp  │      IND      │     1.4 │
│ Maple Pension Trust   │      PEN      │       1 │
│ GlobalTech Solutions  │      TEC      │     1.4 │
│ Riverside Finance Ltd │      FIN      │     1.4 │
└───────────────────────┴───────────────┴─────────┘


#### Counterparty Risk Weights

In [14]:
# ============================================================
# RISK WEIGHTS FOR SCVA
# ============================================================

# Create risk weight table showing counterparty details
df_rw_scva = df_counterparties[["counterparty_name", "sector_code", "credit_quality", "risk_weight"]].copy()

# Display risk weight table
print(format_table(
    df_rw_scva,
    columns=["counterparty_name", "sector_code", "credit_quality", "risk_weight"],
    col_formats={"risk_weight": ".4f"},
    alignments=("left", "center", "center", "right")
))

┌───────────────────────┬───────────────┬──────────────────┬───────────────┐
│ counterparty_name     │  sector_code  │  credit_quality  │   risk_weight │
├───────────────────────┼───────────────┼──────────────────┼───────────────┤
│ Alpha Bank Ltd        │      FIN      │        IG        │          0.05 │
│ Northern Energy Corp  │      IND      │        IG        │          0.03 │
│ Maple Pension Trust   │      PEN      │        IG        │         0.035 │
│ GlobalTech Solutions  │      TEC      │        HY        │         0.055 │
│ Riverside Finance Ltd │      FIN      │        IG        │          0.05 │
└───────────────────────┴───────────────┴──────────────────┴───────────────┘


#### Maturity 

In [17]:
# ============================================================
# MATURITY (M_NS)
# ============================================================

# Create maturity table
df_maturity = df_portfolio[["counterparty_name", "exposure_method", "m_ns"]].copy()

# Display maturity table
print(format_table(
    df_maturity,
    columns=["counterparty_name", "exposure_method", "m_ns"],
    col_formats={"m_ns": ".2f"},
    alignments=("left", "center", "right")
))

┌───────────────────────┬───────────────────┬────────┐
│ counterparty_name     │  exposure_method  │   m_ns │
├───────────────────────┼───────────────────┼────────┤
│ Alpha Bank Ltd        │        IMM        │   4.85 │
│ Northern Energy Corp  │      SA-CCR       │   1.61 │
│ Maple Pension Trust   │        IMM        │   18.5 │
│ GlobalTech Solutions  │      SA-CCR       │      1 │
│ Riverside Finance Ltd │        SFT        │    0.1 │
└───────────────────────┴───────────────────┴────────┘


#### Discount Factor

In [16]:
# ============================================================
# DISCOUNT FACTOR (DF_NS)
# ============================================================

# Step 1: Calculate DF using formula
r = SUPERVISORY_RATE
df_portfolio["df_formula"] = (
    (1 - np.exp(-r * df_portfolio["m_ns"])) / (r * df_portfolio["m_ns"])
)

# Step 2: Apply IMM override (DF = 1.0 for IMM firms only)
df_portfolio["df_ns"] = np.where(
    df_portfolio["exposure_method"] == "IMM",
    1.0,
    df_portfolio["df_formula"]
)

# Create discount factor table
df_discount = df_portfolio[["counterparty_name", "exposure_method", "m_ns", "df_ns"]].copy()

# Display discount factor table
print(format_table(
    df_discount,
    columns=["counterparty_name", "exposure_method", "m_ns", "df_ns"],
    col_formats={"m_ns": ".2f", "df_ns": ".4f"},
    alignments=("left", "center", "right", "right")
))

┌───────────────────────┬───────────────────┬────────┬─────────┐
│ counterparty_name     │  exposure_method  │   m_ns │   df_ns │
├───────────────────────┼───────────────────┼────────┼─────────┤
│ Alpha Bank Ltd        │        IMM        │   4.85 │       1 │
│ Northern Energy Corp  │      SA-CCR       │   1.61 │  0.9608 │
│ Maple Pension Trust   │        IMM        │   18.5 │       1 │
│ GlobalTech Solutions  │      SA-CCR       │      1 │  0.9754 │
│ Riverside Finance Ltd │        SFT        │    0.1 │  0.9975 │
└───────────────────────┴───────────────────┴────────┴─────────┘


#### EAD

In [18]:
# ============================================================
# EXPOSURE AT DEFAULT (EAD_NS)
# ============================================================

# Add product type column (Derivatives for IMM/SA-CCR, SFT for SFT)
df_portfolio["product_type"] = np.where(
    df_portfolio["exposure_method"] == "SFT",
    "SFT",
    "Derivatives"
)

# Create EAD table
df_ead = df_portfolio[["counterparty_name", "product_type", "ead_ns"]].copy()

# Display EAD table
print(format_table(
    df_ead,
    columns=["counterparty_name", "product_type", "ead_ns"],
    col_formats={"ead_ns": ",.0f"},
    alignments=("left", "center", "right")
))

┌───────────────────────┬────────────────┬──────────┐
│ counterparty_name     │  product_type  │   ead_ns │
├───────────────────────┼────────────────┼──────────┤
│ Alpha Bank Ltd        │  Derivatives   │    1,500 │
│ Northern Energy Corp  │  Derivatives   │      450 │
│ Maple Pension Trust   │  Derivatives   │      800 │
│ GlobalTech Solutions  │  Derivatives   │      250 │
│ Riverside Finance Ltd │      SFT       │    2,000 │
└───────────────────────┴────────────────┴──────────┘


#### SCVA

In [52]:
# ============================================================
# SCVA CALCULATION
# ============================================================
# Formula: SCVA = (1/α) × RW × M_NS × EAD_NS × DF_NS

# Calculate SCVA for each netting set
df_portfolio["scva_ns"] = (1 / df_portfolio["alpha"]) * df_portfolio["risk_weight"] * df_portfolio["m_ns"] * df_portfolio["ead_ns"] * df_portfolio["df_ns"]

# Create SCVA table
df_scva_display = df_portfolio[["counterparty_name", "alpha", "risk_weight", "m_ns", "df_ns", "ead_ns", "scva_ns"]].copy()

# Display SCVA table
print(format_table(
    df_scva_display,
    columns=["counterparty_name", "alpha", "risk_weight", "m_ns", "df_ns", "ead_ns", "scva_ns"],
    col_formats={"alpha": ".2f", "risk_weight": ".4f", "m_ns": ".2f", "df_ns": ".4f", "ead_ns": ",.0f", "scva_ns": ",.2f"},
    alignments=("left", "right", "right", "right", "right", "right", "right")
))

┌───────────────────────┬─────────┬───────────────┬────────┬─────────┬──────────┬───────────┐
│ counterparty_name     │   alpha │   risk_weight │   m_ns │   df_ns │   ead_ns │   scva_ns │
├───────────────────────┼─────────┼───────────────┼────────┼─────────┼──────────┼───────────┤
│ Alpha Bank Ltd        │     1.4 │          0.05 │   4.85 │       1 │    1,500 │    259.82 │
│ Northern Energy Corp  │     1.4 │          0.03 │   1.61 │  0.9608 │      450 │     14.92 │
│ Maple Pension Trust   │       1 │         0.035 │   18.5 │       1 │      800 │       518 │
│ GlobalTech Solutions  │     1.4 │         0.055 │      1 │  0.9754 │      250 │      9.58 │
│ Riverside Finance Ltd │     1.4 │          0.05 │    0.1 │  0.9975 │    2,000 │      7.13 │
└───────────────────────┴─────────┴───────────────┴────────┴─────────┴──────────┴───────────┘


### K Reduced

In [20]:
# ============================================================
# K_REDUCED CALCULATION (UNHEDGED)
# ============================================================
# Formula: K_reduced = √[(ρ × Σ SCVA_c)² + (1-ρ²) × Σ SCVA_c²]

# Step 1: Sum of SCVA
sum_scva = df_portfolio["scva_ns"].sum()

# Step 2: Sum of SCVA squared
df_portfolio["scva_ns_squared"] = df_portfolio["scva_ns"] ** 2
sum_scva_squared = df_portfolio["scva_ns_squared"].sum()

# Step 3: Systematic component = (ρ × Σ SCVA)²
systematic_reduced = (RHO * sum_scva) ** 2

# Step 4: Idiosyncratic component = (1 - ρ²) × Σ SCVA²
idiosyncratic_reduced = (1 - RHO**2) * sum_scva_squared

# Step 5: K_reduced = √(Systematic + Idiosyncratic)
k_reduced = np.sqrt(systematic_reduced + idiosyncratic_reduced)

# Create summary table
df_k_reduced_summary = pd.DataFrame({
    "Component": [
        "Σ SCVA_c",
        "Σ SCVA_c²",
        "ρ (correlation)",
        "Systematic (ρ × Σ SCVA)²",
        "Idiosyncratic (1-ρ²) × Σ SCVA²",
        "K_reduced"
    ],
    "Value": [
        sum_scva,
        sum_scva_squared,
        RHO,
        systematic_reduced,
        idiosyncratic_reduced,
        k_reduced
    ]
})

# Display K_reduced summary table
print(format_table(
    df_k_reduced_summary,
    columns=["Component", "Value"],
    col_formats={"Value": ",.2f"},
    alignments=("left", "right")
))

┌────────────────────────────────┬────────────┐
│ Component                      │      Value │
├────────────────────────────────┼────────────┤
│ Σ SCVA_c                       │     809.44 │
│ Σ SCVA_c²                      │ 336,196.22 │
│ ρ (correlation)                │       0.50 │
│ Systematic (ρ × Σ SCVA)²       │ 163,799.47 │
│ Idiosyncratic (1-ρ²) × Σ SCVA² │ 252,147.16 │
│ K_reduced                      │     644.94 │
└────────────────────────────────┴────────────┘


### K Hedged

#### Single Name Hedges (SNH)

##### Correlation Factor

In [ ]:
# ============================================================
# SNH CORRELATION FACTOR (r_hc)
# ============================================================

# Look up correlation from HEDGE_CORRELATIONS table
df_sn_hedges["r_hc"] = df_sn_hedges["hedge_type"].map(HEDGE_CORRELATIONS)

# Merge with counterparty names
df_snh_correlation = df_sn_hedges.merge(
    df_counterparties[["counterparty_id", "counterparty_name"]],
    left_on="hedged_counterparty_id",
    right_on="counterparty_id",
    how="left"
)

# Create correlation factor table
df_corr_display = df_snh_correlation[["hedge_id", "counterparty_name", "hedge_type", "r_hc"]].copy()

# Display correlation factor table
print(format_table(
    df_corr_display,
    columns=["hedge_id", "counterparty_name", "hedge_type", "r_hc"],
    col_formats={"r_hc": ".2f"},
    alignments=("left", "left", "left", "right")
))

##### Hedge Risk Weights

In [22]:
# ============================================================
# SNH RISK WEIGHT (rw_h)
# ============================================================

# Look up risk weight for hedge reference entity
df_sn_hedges["rw_h"] = df_sn_hedges.apply(
    lambda r: RISK_WEIGHTS[r["reference_sector"]][r["reference_cq"]], axis=1
)

# Merge with counterparty data to show both counterparty RW and hedge RW
df_snh_rw = df_snh_correlation.merge(
    df_counterparties[["counterparty_id", "sector_code", "credit_quality", "risk_weight"]],
    on="counterparty_id",
    how="left"
)

# Add hedge risk weight
df_snh_rw["rw_h"] = df_sn_hedges["rw_h"]

# Create risk weight comparison table
df_rw_display = df_snh_rw[["hedge_id", "counterparty_name", "sector_code", "credit_quality", "risk_weight", "rw_h"]].copy()

# Display risk weight table
print(format_table(
    df_rw_display,
    columns=["hedge_id", "counterparty_name", "sector_code", "credit_quality", "risk_weight", "rw_h"],
    col_formats={"risk_weight": ".4f", "rw_h": ".4f"},
    alignments=("left", "left", "center", "center", "right", "right")
))

┌────────────┬───────────────────────┬───────────────┬──────────────────┬───────────────┬────────┐
│ hedge_id   │ counterparty_name     │  sector_code  │  credit_quality  │   risk_weight │   rw_h │
├────────────┼───────────────────────┼───────────────┼──────────────────┼───────────────┼────────┤
│ SN001      │ Alpha Bank Ltd        │      FIN      │        IG        │          0.05 │   0.05 │
│ SN002      │ Alpha Bank Ltd        │      FIN      │        IG        │          0.05 │   0.05 │
│ SN003      │ Northern Energy Corp  │      IND      │        IG        │          0.03 │   0.03 │
│ SN004      │ GlobalTech Solutions  │      TEC      │        HY        │         0.055 │  0.055 │
│ SN005      │ Riverside Finance Ltd │      FIN      │        IG        │          0.05 │   0.05 │
└────────────┴───────────────────────┴───────────────┴──────────────────┴───────────────┴────────┘


##### Maturity

In [23]:
# ============================================================
# SNH MATURITY (M_h)
# ============================================================

# Create maturity table
df_snh_maturity = df_sn_hedges[["hedge_id", "maturity_m_sn"]].copy()

# Display maturity table
print(format_table(
    df_snh_maturity,
    columns=["hedge_id", "maturity_m_sn"],
    col_formats={"maturity_m_sn": ".2f"},
    alignments=("left", "right")
))

┌────────────┬─────────────────┐
│ hedge_id   │   maturity_m_sn │
├────────────┼─────────────────┤
│ SN001      │               3 │
│ SN002      │             2.5 │
│ SN003      │               2 │
│ SN004      │             1.5 │
│ SN005      │             0.5 │
└────────────┴─────────────────┘


##### Discount Factor

In [24]:
# ============================================================
# SNH DISCOUNT FACTOR (DF_h)
# ============================================================
# Formula: DF_h = (1 - exp(-0.05 × M_h)) / (0.05 × M_h)
# Note: Hedges always use the formula (never IMM override)

# Calculate discount factor for each hedge
r = SUPERVISORY_RATE
df_sn_hedges["df_h"] = (
    (1 - np.exp(-r * df_sn_hedges["maturity_m_sn"])) / (r * df_sn_hedges["maturity_m_sn"])
)

# Create discount factor table
df_snh_discount = df_sn_hedges[["hedge_id", "maturity_m_sn", "df_h"]].copy()

# Display discount factor table
print(format_table(
    df_snh_discount,
    columns=["hedge_id", "maturity_m_sn", "df_h"],
    col_formats={"maturity_m_sn": ".2f", "df_h": ".4f"},
    alignments=("left", "right", "right")
))

┌────────────┬─────────────────┬────────┐
│ hedge_id   │   maturity_m_sn │   df_h │
├────────────┼─────────────────┼────────┤
│ SN001      │               3 │ 0.9286 │
│ SN002      │             2.5 │   0.94 │
│ SN003      │               2 │ 0.9516 │
│ SN004      │             1.5 │ 0.9634 │
│ SN005      │             0.5 │ 0.9876 │
└────────────┴─────────────────┴────────┘


##### Notional

In [25]:
# ============================================================
# SNH NOTIONAL (B_h)
# ============================================================

# Create notional table
df_snh_notional = df_sn_hedges[["hedge_id", "notional_b_sn"]].copy()

# Display notional table
print(format_table(
    df_snh_notional,
    columns=["hedge_id", "notional_b_sn"],
    col_formats={"notional_b_sn": ",.0f"},
    alignments=("left", "right")
))

┌────────────┬─────────────────┐
│ hedge_id   │   notional_b_sn │
├────────────┼─────────────────┤
│ SN001      │             500 │
│ SN002      │             300 │
│ SN003      │             200 │
│ SN004      │             150 │
│ SN005      │           1,000 │
└────────────┴─────────────────┘


##### SNH 

In [26]:
# ============================================================
# SNH CALCULATION (SINGLE-NAME HEDGE BENEFIT)
# ============================================================
# Formula: SNH = r_hc × RW_h × M_h × B_h × DF_h

# Step 1: Calculate hedge value = RW_h × M_h × B_h × DF_h
df_sn_hedges["hedge_value"] = (
    df_sn_hedges["rw_h"] * 
    df_sn_hedges["maturity_m_sn"] * 
    df_sn_hedges["notional_b_sn"] * 
    df_sn_hedges["df_h"]
)

# Step 2: Apply correlation r_hc to get SNH
df_sn_hedges["snh_component"] = df_sn_hedges["r_hc"] * df_sn_hedges["hedge_value"]

# Create SNH table showing all components
df_snh_calc = df_sn_hedges[["hedge_id", "r_hc", "rw_h", "maturity_m_sn", "notional_b_sn", "df_h", "snh_component"]].copy()

# Display SNH calculation table
print(format_table(
    df_snh_calc,
    columns=["hedge_id", "r_hc", "rw_h", "maturity_m_sn", "notional_b_sn", "df_h", "snh_component"],
    col_formats={"r_hc": ".2f", "rw_h": ".4f", "maturity_m_sn": ".2f", "notional_b_sn": ",.0f", "df_h": ".4f", "snh_component": ",.2f"},
    alignments=("left", "right", "right", "right", "right", "right", "right")
))

┌────────────┬────────┬────────┬─────────────────┬─────────────────┬────────┬─────────────────┐
│ hedge_id   │   r_hc │   rw_h │   maturity_m_sn │   notional_b_sn │   df_h │   snh_component │
├────────────┼────────┼────────┼─────────────────┼─────────────────┼────────┼─────────────────┤
│ SN001      │      1 │   0.05 │               3 │             500 │ 0.9286 │           69.65 │
│ SN002      │    0.8 │   0.05 │             2.5 │             300 │   0.94 │            28.2 │
│ SN003      │    0.5 │   0.03 │               2 │             200 │ 0.9516 │            5.71 │
│ SN004      │    0.8 │  0.055 │             1.5 │             150 │ 0.9634 │            9.54 │
│ SN005      │      1 │   0.05 │             0.5 │           1,000 │ 0.9876 │           24.69 │
└────────────┴────────┴────────┴─────────────────┴─────────────────┴────────┴─────────────────┘


#### Hedge Mismatch Adjustment (HMA)

In [ ]:
# ============================================================
# HMA CALCULATION (HEDGE MISMATCH ADD-ON)
# ============================================================
# Formula: HMA = (1 - r_hc²) × (RW_h × M_h × B_h × DF_h)²
# This penalizes proxy hedges where r_hc < 100%

# Step 1: Calculate HMA factor = (1 - r_hc²)
df_sn_hedges["hma_factor"] = 1 - df_sn_hedges["r_hc"] ** 2

# Step 2: Calculate squared hedge value
df_sn_hedges["hedge_value_squared"] = df_sn_hedges["hedge_value"] ** 2

# Step 3: Calculate HMA component
df_sn_hedges["hma_component"] = df_sn_hedges["hma_factor"] * df_sn_hedges["hedge_value_squared"]

# Create HMA table showing all components
df_hma_calc = df_sn_hedges[["hedge_id", "r_hc", "rw_h", "maturity_m_sn", "notional_b_sn", "df_h", "hma_component"]].copy()

# Display HMA calculation table
print(format_table(
    df_hma_calc,
    columns=["hedge_id", "r_hc", "rw_h", "maturity_m_sn", "notional_b_sn", "df_h", "hma_component"],
    col_formats={"r_hc": ".2f", "rw_h": ".4f", "maturity_m_sn": ".2f", "notional_b_sn": ",.0f", "df_h": ".4f", "hma_component": ",.2f"},
    alignments=("left", "right", "right", "right", "right", "right", "right")
))

#### Index Hedges (IH)

##### Index Risk Weights

In [29]:
# ============================================================
# INDEX HEDGE RISK WEIGHT CALCULATION
# ============================================================
# Index RW = (Σ weight_i × RW_i) × 0.7

# IH001: Index Hedge 1 composition
df_ih001_constituents = pd.DataFrame({
    "sector_code": ["FIN", "IND", "CON", "TEC", "HLT", "OTH"],
    "credit_quality": ["IG", "IG", "IG", "IG", "IG", "IG"],
    "weight": [0.248, 0.200, 0.200, 0.104, 0.152, 0.096]
})

# Look up risk weight for each sector
df_ih001_constituents["risk_weight"] = df_ih001_constituents.apply(
    lambda r: RISK_WEIGHTS[r["sector_code"]][r["credit_quality"]], axis=1
)

# Calculate weighted contribution
df_ih001_constituents["weighted_rw"] = df_ih001_constituents["weight"] * df_ih001_constituents["risk_weight"]

# Sum to get pre-scalar RW for IH001
rw_ih001_pre_scalar = df_ih001_constituents["weighted_rw"].sum()

# Add total row
df_ih001_display = df_ih001_constituents.copy()
df_ih001_display.loc[len(df_ih001_display)] = ["TOTAL", "", "", "", rw_ih001_pre_scalar]

# Display IH001 table
print("IH001 - Index Hedge 1")
print(format_table(
    df_ih001_display,
    columns=["sector_code", "credit_quality", "risk_weight", "weight", "weighted_rw"],
    col_formats={"risk_weight": ".4f", "weight": ".3f", "weighted_rw": ".6f"},
    alignments=("center", "center", "right", "right", "right")
))

print("\n")

# IH002: Index Hedge 2 composition
df_ih002_constituents = pd.DataFrame({
    "sector_code": ["FIN", "IND", "CON", "TEC", "HLT", "OTH"],
    "credit_quality": ["IG", "IG", "IG", "IG", "IG", "IG"],
    "weight": [0.200, 0.248, 0.152, 0.200, 0.104, 0.096]
})

# Look up risk weight for each sector
df_ih002_constituents["risk_weight"] = df_ih002_constituents.apply(
    lambda r: RISK_WEIGHTS[r["sector_code"]][r["credit_quality"]], axis=1
)

# Calculate weighted contribution
df_ih002_constituents["weighted_rw"] = df_ih002_constituents["weight"] * df_ih002_constituents["risk_weight"]

# Sum to get pre-scalar RW for IH002
rw_ih002_pre_scalar = df_ih002_constituents["weighted_rw"].sum()

# Add total row
df_ih002_display = df_ih002_constituents.copy()
df_ih002_display.loc[len(df_ih002_display)] = ["TOTAL", "", "", "", rw_ih002_pre_scalar]

# Display IH002 table
print("IH002 - Index Hedge 2")
print(format_table(
    df_ih002_display,
    columns=["sector_code", "credit_quality", "risk_weight", "weight", "weighted_rw"],
    col_formats={"risk_weight": ".4f", "weight": ".3f", "weighted_rw": ".6f"},
    alignments=("center", "center", "right", "right", "right")
))


IH001 - Index Hedge 1
┌───────────────┬──────────────────┬───────────────┬──────────┬───────────────┐
│  sector_code  │  credit_quality  │   risk_weight │   weight │   weighted_rw │
├───────────────┼──────────────────┼───────────────┼──────────┼───────────────┤
│      FIN      │        IG        │        0.0500 │    0.248 │        0.0124 │
│      IND      │        IG        │        0.0300 │    0.200 │         0.006 │
│      CON      │        IG        │        0.0300 │    0.200 │         0.006 │
│      TEC      │        IG        │        0.0200 │    0.104 │       0.00208 │
│      HLT      │        IG        │        0.0150 │    0.152 │       0.00228 │
│      OTH      │        IG        │        0.0500 │    0.096 │        0.0048 │
│     TOTAL     │                  │               │          │       0.03356 │
└───────────────┴──────────────────┴───────────────┴──────────┴───────────────┘


IH002 - Index Hedge 2
┌───────────────┬──────────────────┬───────────────┬──────────┬───────────

##### Scalar

In [30]:
# ============================================================
# INDEX HEDGE SCALAR APPLICATION
# ============================================================

# Apply 0.7 index scalar
rw_ih001 = rw_ih001_pre_scalar * INDEX_SCALAR
rw_ih002 = rw_ih002_pre_scalar * INDEX_SCALAR

# Create scalar table
df_index_scalar = pd.DataFrame({
    "index_id": ["IH001", "IH002"],
    "weighted_rw": [rw_ih001_pre_scalar, rw_ih002_pre_scalar],
    "scalar": [INDEX_SCALAR, INDEX_SCALAR],
    "rw_ind": [rw_ih001, rw_ih002]
})

# Store final risk weights in index_hedges dataframe
df_index_hedges["rw_ind"] = [rw_ih001, rw_ih002]

# Display scalar table
print(format_table(
    df_index_scalar,
    columns=["index_id", "weighted_rw", "scalar", "rw_ind"],
    col_formats={"weighted_rw": ".6f", "scalar": ".2f", "rw_ind": ".6f"},
    alignments=("left", "right", "right", "right")
))

┌────────────┬───────────────┬──────────┬──────────┐
│ index_id   │   weighted_rw │   scalar │   rw_ind │
├────────────┼───────────────┼──────────┼──────────┤
│ IH001      │       0.03356 │      0.7 │ 0.023492 │
│ IH002      │       0.03236 │      0.7 │ 0.022652 │
└────────────┴───────────────┴──────────┴──────────┘


##### Maturity

In [31]:
# ============================================================
# INDEX HEDGE MATURITY (M_ind)
# ============================================================

# Create maturity table
df_ih_maturity = df_index_hedges[["index_id", "maturity_m_ind"]].copy()

# Display maturity table
print(format_table(
    df_ih_maturity,
    columns=["index_id", "maturity_m_ind"],
    col_formats={"maturity_m_ind": ".2f"},
    alignments=("left", "right")
))

┌────────────┬──────────────────┐
│ index_id   │   maturity_m_ind │
├────────────┼──────────────────┤
│ IH001      │                5 │
│ IH002      │                3 │
└────────────┴──────────────────┘


##### Discount Factor

In [32]:
# ============================================================
# INDEX HEDGE DISCOUNT FACTOR (DF_ind)
# ============================================================
# Formula: DF_ind = (1 - exp(-0.05 × M_ind)) / (0.05 × M_ind)

# Calculate discount factor for each index hedge
r = SUPERVISORY_RATE
df_index_hedges["df_ind"] = (
    (1 - np.exp(-r * df_index_hedges["maturity_m_ind"])) / (r * df_index_hedges["maturity_m_ind"])
)

# Create discount factor table
df_ih_discount = df_index_hedges[["index_id", "maturity_m_ind", "df_ind"]].copy()

# Display discount factor table
print(format_table(
    df_ih_discount,
    columns=["index_id", "maturity_m_ind", "df_ind"],
    col_formats={"maturity_m_ind": ".2f", "df_ind": ".4f"},
    alignments=("left", "right", "right")
))

┌────────────┬──────────────────┬──────────┐
│ index_id   │   maturity_m_ind │   df_ind │
├────────────┼──────────────────┼──────────┤
│ IH001      │                5 │   0.8848 │
│ IH002      │                3 │   0.9286 │
└────────────┴──────────────────┴──────────┘


##### Notional

In [33]:
# ============================================================
# INDEX HEDGE NOTIONAL (B_ind)
# ============================================================

# Create notional table
df_ih_notional = df_index_hedges[["index_id", "notional_b_ind"]].copy()

# Display notional table
print(format_table(
    df_ih_notional,
    columns=["index_id", "notional_b_ind"],
    col_formats={"notional_b_ind": ",.0f"},
    alignments=("left", "right")
))

┌────────────┬──────────────────┐
│ index_id   │   notional_b_ind │
├────────────┼──────────────────┤
│ IH001      │              750 │
│ IH002      │              500 │
└────────────┴──────────────────┘


##### IH

In [34]:
# ============================================================
# IH CALCULATION (INDEX HEDGE BENEFIT)
# ============================================================
# Formula: IH = RW_ind × M_ind × B_ind × DF_ind

# Calculate IH component for each index hedge
df_index_hedges["ih_component"] = (
    df_index_hedges["rw_ind"] * 
    df_index_hedges["maturity_m_ind"] * 
    df_index_hedges["notional_b_ind"] * 
    df_index_hedges["df_ind"]
)

# Create IH table with individual components
df_ih_calc = df_index_hedges[["index_id", "rw_ind", "maturity_m_ind", "notional_b_ind", "df_ind", "ih_component"]].copy()

# Add total row
total_ih = df_index_hedges["ih_component"].sum()
df_ih_calc.loc[len(df_ih_calc)] = ["Total IH", "", "", "", "", total_ih]

# Display IH calculation table
print(format_table(
    df_ih_calc,
    columns=["index_id", "rw_ind", "maturity_m_ind", "notional_b_ind", "df_ind", "ih_component"],
    col_formats={"rw_ind": ".6f", "maturity_m_ind": ".2f", "notional_b_ind": ",.0f", "df_ind": ".4f", "ih_component": ",.2f"},
    alignments=("left", "right", "right", "right", "right", "right")
))

┌────────────┬──────────┬──────────────────┬──────────────────┬──────────┬────────────────┐
│ index_id   │   rw_ind │   maturity_m_ind │   notional_b_ind │   df_ind │   ih_component │
├────────────┼──────────┼──────────────────┼──────────────────┼──────────┼────────────────┤
│ IH001      │ 0.023492 │             5.00 │              750 │   0.8848 │          77.95 │
│ IH002      │ 0.022652 │             3.00 │              500 │   0.9286 │          31.55 │
│ Total IH   │          │                  │                  │          │          109.5 │
└────────────┴──────────┴──────────────────┴──────────────────┴──────────┴────────────────┘


#### K Hedged

In [ ]:
# ============================================================
# K_HEDGED CALCULATION - PREPARATION
# ============================================================

# Aggregate SNH and HMA by counterparty
df_snh_by_cpty = df_sn_hedges.groupby("hedged_counterparty_id").agg({
    "snh_component": "sum",
    "hma_component": "sum"
}).reset_index()
df_snh_by_cpty.columns = ["counterparty_id", "snh_c", "hma_c"]

# Merge SCVA with SNH/HMA
df_hedged = df_portfolio[["counterparty_id", "counterparty_name", "scva_ns"]].copy()
df_hedged = df_hedged.merge(df_snh_by_cpty, on="counterparty_id", how="left")

# Fill NaN with 0 (counterparties without hedges)
df_hedged["snh_c"] = df_hedged["snh_c"].fillna(0)
df_hedged["hma_c"] = df_hedged["hma_c"].fillna(0)

##### Systematic Component

In [41]:
# ============================================================
# SYSTEMATIC COMPONENT
# ============================================================
# Formula: Systematic = (ρ × Σ SCVA - ρ × Σ SNH - IH)²
#        = (ρ × Σ(SCVA - SNH) - IH)²

# Step 1: Sum of SCVA
sum_scva_hedged = df_hedged["scva_ns"].sum()

# Step 2: Sum of SNH
sum_snh = df_hedged["snh_c"].sum()

# Step 3: Calculate systematic component
systematic_hedged = (RHO * sum_scva_hedged - RHO * sum_snh - total_ih) ** 2

# Create systematic table
df_systematic = pd.DataFrame({
    "Component": [
        "Σ SCVA",
        "Σ SNH",
        "Σ IH",
        "ρ (correlation)",
        "Systematic (ρ (SCVA - SNH) - IH))²"
    ],
    "Value": [
        sum_scva_hedged,
        sum_snh,
        total_ih,
        RHO,
        systematic_hedged
    ]
})

# Display systematic table
print(format_table(
    df_systematic,
    columns=["Component", "Value"],
    col_formats={"Value": ",.2f"},
    alignments=("left", "right")
))

┌────────────────────────────────────┬───────────┐
│ Component                          │     Value │
├────────────────────────────────────┼───────────┤
│ Σ SCVA                             │    809.44 │
│ Σ SNH                              │    137.78 │
│ Σ IH                               │    109.50 │
│ ρ (correlation)                    │      0.50 │
│ Systematic (ρ (SCVA - SNH) - IH))² │ 51,225.56 │
└────────────────────────────────────┴───────────┘


##### Idiosyncratic Component

In [45]:
# ============================================================
# IDIOSYNCRATIC COMPONENT
# ============================================================
# Formula: Idiosyncratic = (1-ρ²) × Σ(SCVA - SNH)²

# Step 1: Calculate SCVA sum
sum_scva_idio = df_hedged["scva_ns"].sum()

# Step 2: Calculate SNH sum
sum_snh_idio = df_hedged["snh_c"].sum()

# Step 3: Calculate (SCVA - SNH) components
df_hedged["scva_minus_snh"] = df_hedged["scva_ns"] - df_hedged["snh_c"]
df_hedged["scva_minus_snh_squared"] = df_hedged["scva_minus_snh"] ** 2
sum_scva_minus_snh_squared = df_hedged["scva_minus_snh_squared"].sum()

# Step 4: Calculate idiosyncratic component
factor_1_minus_rho_squared = 1 - RHO**2
idiosyncratic_hedged = factor_1_minus_rho_squared * sum_scva_minus_snh_squared

# Create detailed idiosyncratic table
df_idiosyncratic = pd.DataFrame({
    "Component": [
        "Σ SCVA",
        "Σ SNH",
        "Σ (SCVA - SNH)",
        "Σ (SCVA - SNH)²",
        "(1 - ρ²)",
        "Idiosyncratic (1-ρ²) × Σ(SCVA - SNH)²"
    ],
    "Value": [
        sum_scva_idio,
        sum_snh_idio,
        sum_scva_idio - sum_snh_idio,
        sum_scva_minus_snh_squared,
        factor_1_minus_rho_squared,
        idiosyncratic_hedged
    ]
})

# Display idiosyncratic table
print(format_table(
    df_idiosyncratic,
    columns=["Component", "Value"],
    col_formats={"Value": ",.2f"},
    alignments=("left", "right")
))

┌───────────────────────────────────────┬────────────┐
│ Component                             │      Value │
├───────────────────────────────────────┼────────────┤
│ Σ SCVA                                │     809.44 │
│ Σ SNH                                 │     137.78 │
│ Σ (SCVA - SNH)                        │     671.66 │
│ Σ (SCVA - SNH)²                       │ 294,953.09 │
│ (1 - ρ²)                              │       0.75 │
│ Idiosyncratic (1-ρ²) × Σ(SCVA - SNH)² │ 221,214.82 │
└───────────────────────────────────────┴────────────┘


##### Hedge Mismatch Total

In [ ]:
# ============================================================
# HEDGE MISMATCH TOTAL
# ============================================================

# Sum of HMA across all counterparties
sum_hma = df_hedged["hma_c"].sum()

# Create HMA table
df_hma_total = pd.DataFrame({
    "Component": ["Σ HMA"],
    "Value": [sum_hma]
})

# Display HMA table
print(format_table(
    df_hma_total,
    columns=["Component", "Value"],
    col_formats={"Value": ",.2f"},
    alignments=("left", "right")
))

##### K Hedged Calculation

In [ ]:
# ============================================================
# K_HEDGED FINAL CALCULATION
# ============================================================
# Formula: K_hedged = √(Systematic + Idiosyncratic + HMA)

# Step 1: Sum the three components
total_before_sqrt = systematic_hedged + idiosyncratic_hedged + sum_hma

# Step 2: Take square root to get K_hedged
k_hedged = np.sqrt(total_before_sqrt)

# Create K_hedged summary table
df_k_hedged_summary = pd.DataFrame({
    "Component": [
        "Systematic",
        "Idiosyncratic",
        "HMA",
        "Total (Systematic + Idiosyncratic + HMA)",
        "K_hedged (√Total)"
    ],
    "Value": [
        systematic_hedged,
        idiosyncratic_hedged,
        sum_hma,
        total_before_sqrt,
        k_hedged
    ]
})

# Display K_hedged summary table
print(format_table(
    df_k_hedged_summary,
    columns=["Component", "Value"],
    col_formats={"Value": ",.2f"},
    alignments=("left", "right")
))

### K Full

In [ ]:
# ============================================================
# K_FULL CALCULATION
# ============================================================
# Formula: K_full = β × K_reduced + (1-β) × K_hedged

# Step 1: Calculate K_full
k_full = BETA * k_reduced + (1 - BETA) * k_hedged

# Create K_full table
df_k_full = pd.DataFrame({
    "Component": [
        "K_reduced",
        "K_hedged",
        "β (hedge floor)",
        "K_full = β × K_reduced + (1-β) × K_hedged"
    ],
    "Value": [
        k_reduced,
        k_hedged,
        BETA,
        k_full
    ]
})

# Display K_full table
print(format_table(
    df_k_full,
    columns=["Component", "Value"],
    col_formats={"Value": ",.2f"},
    alignments=("left", "right")
))

### K Final

In [51]:
# ============================================================
# K_FINAL CALCULATION (BA-CVA CAPITAL)
# ============================================================
# Formula: K_final = DS_BA_CVA × K_full

# Step 1: Calculate K_final (BA-CVA Capital Requirement)
k_final = DS_BA_CVA * k_full

# Create K_final table
df_k_final = pd.DataFrame({
    "Component": [
        "K_full",
        "DS_BA_CVA (discount scalar)",
        "K_final = DS_BA_CVA × K_full"
    ],
    "Value": [
        k_full,
        DS_BA_CVA,
        k_final
    ]
})

# Display K_final table
print(format_table(
    df_k_final,
    columns=["Component", "Value"],
    col_formats={"Value": ",.2f"},
    alignments=("left", "right")
))

┌──────────────────────────────┬─────────┐
│ Component                    │   Value │
├──────────────────────────────┼─────────┤
│ K_full                       │  553.13 │
│ DS_BA_CVA (discount scalar)  │    0.65 │
│ K_final = DS_BA_CVA × K_full │  359.54 │
└──────────────────────────────┴─────────┘
